In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

Mounted at /content/drive


Step 1: Importing the required libraries

In [ ]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules


Step 2: Loading and exploring the data

In [ ]:
# Loading the Data
data = pd.read_excel('Online_Retail.xlsx')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
# Exploring the columns of the data
data.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [ ]:
# Exploring the different regions of transactions
data.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

Step 3: Cleaning the Data

In [ ]:
# Stripping extra spaces in the description
data['Description'] = data['Description'].str.strip()

# Dropping the rows without any invoice number
data.dropna(axis = 0, subset =['InvoiceNo'], inplace = True)
data['InvoiceNo'] = data['InvoiceNo'].astype('str')

# Dropping all transactions which were done on credit
data = data[~data['InvoiceNo'].str.contains('C')]

Step 4: Splitting the data according to the region of transaction

In [ ]:
# Transactions done in France
basket_France = (data[data['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

# Transactions done in the United Kingdom
basket_UK = (data[data['Country'] =="United Kingdom"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

# Transactions done in Portugal
basket_Por = (data[data['Country'] =="Portugal"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_Sweden = (data[data['Country'] =="Sweden"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [ ]:
print(len(basket_France))

392


Step 5: Hot encoding the Data

In [ ]:
# Defining the hot encoding function to make the data suitable
# for the concerned libraries
def hot_encode(x):
    if(x<= 0):
        return 0
    if(x>= 1):
        return 1

# Encoding the datasets
basket_encoded = basket_France.applymap(hot_encode)
basket_France = basket_encoded

basket_encoded = basket_UK.applymap(hot_encode)
basket_UK = basket_encoded

basket_encoded = basket_Por.applymap(hot_encode)
basket_Por = basket_encoded

basket_encoded = basket_Sweden.applymap(hot_encode)
basket_Sweden = basket_encoded

In [ ]:
print(len(basket_France))

392


Step 6: Building the models and analyzing the results

a) France:

In [ ]:
# Building the model
frq_items = apriori(basket_France, min_support = 0.05, use_colnames = True)

# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
print(rules.head())

                                           antecedents  ... conviction
45                        (JUMBO BAG WOODLAND ANIMALS)  ...        inf
259  (RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...  ...        inf
270  (RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...  ...        inf
301  (SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...  ...  34.897959
302  (SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...  ...  34.489796

[5 rows x 9 columns]


b) United Kingdom:

In [ ]:
frq_items = apriori(basket_UK, min_support = 0.01, use_colnames = True)
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
print(rules.head())

                                       antecedents  ... conviction
116           (BEADED CRYSTAL HEART PINK ON STICK)  ...  39.637371
2020  (SUKI  SHOULDER BAG, JAM MAKING SET PRINTED)  ...  26.096206
2296         (HERB MARKER MINT, HERB MARKER THYME)  ...  21.947227
2300   (HERB MARKER PARSLEY, HERB MARKER ROSEMARY)  ...  20.444951
2302      (HERB MARKER PARSLEY, HERB MARKER THYME)  ...  20.443842

[5 rows x 9 columns]


c) Portugal:

In [ ]:
frq_items = apriori(basket_Por, min_support = 0.05, use_colnames = True)
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
print(rules.head())

                             antecedents  ... conviction
1170  (SET 12 COLOUR PENCILS DOLLY GIRL)  ...        inf
1171    (SET 12 COLOUR PENCILS SPACEBOY)  ...        inf
1172  (SET 12 COLOUR PENCILS DOLLY GIRL)  ...        inf
1173  (SET OF 4 KNICK KNACK TINS LONDON)  ...        inf
1174  (SET 12 COLOUR PENCILS DOLLY GIRL)  ...        inf

[5 rows x 9 columns]


d) Sweden:

In [ ]:
frq_items = apriori(basket_Sweden, min_support = 0.05, use_colnames = True)
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
print(rules.head())

                        antecedents  ... conviction
0     (PACK OF 72 SKULL CAKE CASES)  ...        inf
1     (12 PENCILS SMALL TUBE SKULL)  ...        inf
4           (36 DOILIES DOLLY GIRL)  ...        inf
5    (ASSORTED BOTTLE TOP  MAGNETS)  ...        inf
180  (CHILDRENS CUTLERY DOLLY GIRL)  ...        inf

[5 rows x 9 columns]
